# Using mit-bih-noise-stress-test-database, Test our models and index performances

## Import

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import pickle as pkl
import seaborn as sns
import os
import itertools
import sys
import xarray as xr
sys.path.append(os.path.join(os.getcwd(), ".."))
from operations.dataset_manager import get_path_petastorm_format

In [ ]:
plt.rcParams.update({"font.size": 32})
plt.rcParams["legend.fontsize"] = 32

In [ ]:
d = xr.load_dataset("/workspaces/ecg_evaluation/results/quality_metrics.nc")
print(d)

## Paths

In [ ]:
models_save_path = "/workspaces/ecg_evaluation/results"
data_save_path = get_path_petastorm_format("mit-bih-noise-stress-test-database-1.0.0","ParquetFile")
save_path  = "/workspaces/ecg_evaluation/results"

# Get the dataset

In [ ]:
## get save files (where model is saved)

model_name = [
                f.split(".")[0]
                for f in os.listdir(models_save_path)
                if os.path.isfile(os.path.join(models_save_path,f )) and (f.endswith(".sav"))
            ]

In [ ]:
name_index = [
        "Corr_interlead",
        "Corr_intralead",
        "wPMF",
        "SNRECG",
        "HR",
        "Flatline",
        "TSD",
    ]

# Test the model on new dataset

### Please note that you will have to get your dataset ready and your metrics already calculated using the command describe on the git repo. 

In [ ]:
data_test_path = "/workspaces/ecg_evaluation/results/mit_bih_noise_test_metrics.nc"
metrics = xr.load_dataset(data_test_path)

In [ ]:
## check data metrics 
id_signals = metrics.id.values
values_metrics = metrics.quality_metrics.values
metrics_name = metrics.metric_name.values
signal = metrics.signal.values
nb_segment = metrics.number_signal.values[0]

## Test each model performance for each noise

### We will specifically focus on the noises signal. So, let's isolate them :

In [ ]:
list_noise_new = ["em","ma","bw"]
list_noise_old = ["oldem","oldma","oldbw"]
index_noise_new = [np.where(id_signals==n)[0][0] for n in list_noise_new]
index_noise_old = [np.where(id_signals==n)[0][0] for n in list_noise_old]

In [ ]:
## 3 type of noise. We will reunite the old and new version
noise_data = np.zeros([len(list_noise_new),nb_segment*2,values_metrics.shape[-1]])

for j in range(noise_data.shape[0]):
    noise_data[j,:,:] = np.concatenate((values_metrics[index_noise_new[j],:,:],values_metrics[index_noise_old[j],:,:]),axis = 0)


In [ ]:
## Small check concerning the values performance for each noise.
palette = itertools.cycle(sns.color_palette())
## take the average result obtained over all the segment of all datasets.
trial_rinter = noise_data[2,:,np.where(metrics_name==name_index[0])[0]]

sns.histplot(trial_rinter[0,:],color=next(palette))
print(np.mean(trial_rinter[0,:]))
print(np.std(trial_rinter[0,:]))


### Load model 